In [1]:
import sys
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score
import pandas as pd
from collections import defaultdict

os.environ['PROJECT_PATH'] = "/scratch/hz2103/AutoDataDescription"
os.environ['DATA_PATH'] = "/scratch/hz2103/AutoDataDescription/data/"
os.environ['NTCIR_PATH'] = "/scratch/hz2103/AutoDataDescription/data/ntcir/"
sys.path.append(os.getenv("PROJECT_PATH"))
data_path = os.getenv("DATA_PATH")
ntcir_path = os.getenv("NTCIR_PATH")
# openai_api_key = os.getenv("OPENAI_API_KEY")

from src.generate_description import AutoDescription
from src.process_csv import CsvProcessor
from util.ntcir_util import NtcirUtil
from util.csv_util import CsvUtil
from util.ndcg_util import NDCGCalculator, calculate_cosine_similarity

import warnings
warnings.filterwarnings('ignore')
import json

ecir_path = "../data/ecir/data/"


/home/hz2103/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/hz2103/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/hz2103/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/hz2103/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
meta_1st = {}
meta_2nd = {}
count_2lvl = {}
for data_folder in os.listdir(ecir_path):
    json_path = ecir_path+data_folder+"/meta.json"
    meta_1st[data_folder] = json.load(open(json_path, "r"))
    for sub_data_folder in os.listdir(ecir_path+data_folder):
        if sub_data_folder!='meta.json':
            if data_folder not in count_2lvl:
                count_2lvl[data_folder]=[sub_data_folder]
            else:
                count_2lvl[data_folder].append(sub_data_folder)
            sub_json_path = ecir_path+data_folder+"/"+sub_data_folder+"/meta.json"
            meta_2nd[data_folder+"/"+sub_data_folder] = json.load(open(sub_json_path, "r"))



In [3]:
dataset_single = ['/'.join([k]+count_2lvl[k]) for k in count_2lvl if len(count_2lvl[k])==1]
len(meta_1st), len(meta_2nd), len(count_2lvl), \
sum([len(count_2lvl[k]) for k in count_2lvl if len(count_2lvl[k])==1]), \
len(dataset_single)


(2083, 3095, 2083, 1813, 1813)

In [4]:
def extract_query_rel(qrel_file_path, dataset_single):
    from collections import defaultdict
    query_rel = defaultdict(lambda: defaultdict(list))
    query_rel_list = []
    with open(qrel_file_path, 'r') as f:
        lines = f.readlines()[1:]
        for line in lines:
            t, r, d, _ = line.split(",")
            d = '/'.join(d.split('/')[1:-1])
            r = float(r)
#             if d in dataset_single:
            query_rel[d][t].append(r)
            query_rel_list.append((d, t, r))
    return query_rel, query_rel_list


In [5]:
# query_rel, query_rel_list = extract_query_rel('../data/ecir/concatenated.csv', dataset_single)
query_rel, query_rel_list = extract_query_rel('../data/ecir/concatenated.csv', [])
dataset_info = pd.read_csv("../data/ecir/DatasetInfo.csv")
dataset_info.replace(np.nan, '')
dataset_info

,Task,DatasetID,Title,Relevance,Description
0,Q1,fbeabe3a-f302-446d-a88b-3b81a46d1d03,"Anemometer Data (Wind Speed, Direction) for Be...",2,"Wind data collected from Beloit, Kansas from a..."
1,Q1,cfd52641-d8a8-42bf-84fc-8db1d2ae6e20,"Anemometer Data (Wind Speed, Direction) for Be...",2,"Wind data collected from Beloit, Kansas from a..."
2,Q1,6cdbee9d-fc41-44b9-9a8d-2567422c4445,"Anemometer Data (Wind Speed, Direction) for Be...",2,"Wind data collected from Beloit, Kansas from a..."
3,Q1,ed56f7b7-21ee-47ed-b174-d56155c0e134,"Anemometer Data (Wind Speed, Direction) for Be...",2,"Wind data collected from Beloit, Kansas from a..."
4,Q1,82a1b041-8548-4b2c-a5e1-471900cf952e,"Anemometer Data (Wind Speed, Direction) for Be...",2,"Wind data collected from Beloit, Kansas from a..."
5,Q1,4f94168e-2f48-4192-a246-f60b2f3b23d8,"Anemometer Data (Wind Speed, Direction) for Wa...",1,"Wind data collected from Watertown, South Dako..."
6,Q1,ce45103a-2a0a-4be9-9fe0-ceca48712068,"Anemometer Data (Wind Speed, Direction) for Wa...",1,"Wind data collected from Watertown, South Dako..."
7,Q1,4c344d31-fe88-4f6d-945f-b2469b667163,"Anemometer Data (Wind Speed, Direction) for Wa...",1,"Wind data collected from Watertown, South Dako..."
8,Q1,b6f420e2-0a2d-4377-9501-4f8b50c9dfdf,"Anemometer Data (Wind Speed, Direction) for Wa...",1,"Wind data collected from Watertown, South Dako..."
9,Q1,399f395c-78bd-42cb-8c33-cd30790c431a,"Anemometer Data (Wind Speed, Direction) for Wa...",1,"Wind data collected from Watertown, South Dako..."


In [6]:
# dataset_title = dict(zip(dataset_info["Task"], dataset_info["DatasetID"], dataset_info["Title"]))
dataset_title = {row['DatasetID']: (row['Task'], row['Title'], row['Description'], row['Relevance']) for index, row in dataset_info.iterrows()}
dataset_title

{'fbeabe3a-f302-446d-a88b-3b81a46d1d03': ('Q1',
  'Anemometer Data (Wind Speed, Direction) for Beloit, Kansas (May - Jun, 2003)',
  'Wind data collected from Beloit, Kansas from a 20-meter anemometer as part of the Western Area Power Administration anemometer loan program. Ten-minute average wind speed and direction is available for 2003 and 2004. The data was originally made available by Wind Powering America, a DOE Office of Energy Efficiency & Renewable Energy (EERE) program. A dynamic map displaying all available data from DOE anemometer loan programs is available',
  2),
 'cfd52641-d8a8-42bf-84fc-8db1d2ae6e20': ('Q1',
  'Anemometer Data (Wind Speed, Direction) for Beloit, Kansas (Sep - Nov, 2003)',
  'Wind data collected from Beloit, Kansas from a 20-meter anemometer as part of the Western Area Power Administration anemometer loan program. Ten-minute average wind speed and direction is available for 2003 and 2004. The data was originally made available by Wind Powering America, a 

In [7]:
model_versions = ["gpt4", "gpt3.5", "llama7b", "llama13b", "llama34b"]
prompt_versions = ["p1", "p2", "p3"]
task = "task2"

gold_descriptions = defaultdict(list)
for model_v in model_versions:
    for prompt_v in prompt_versions:
        if model_v=="gpt4" and prompt_v=="p3":
            continue
        experiment_version = "_".join([task, model_v, prompt_v])
        desc_txt = "../data/ecir/"+task+"/"+experiment_version+".txt"

        from collections import defaultdict
        with open(desc_txt, 'r') as f:
            lines = f.readlines()
            for line in lines:
#                 if "\t" in line:
#                 print(line.split("\t")[0])
                gold_descriptions[experiment_version].append(line.split("\t"))
#                 elif "   " in line:
#                     gold_descriptions[experiment_version].append(line.split("   "))
#                 else:
#                     gold_descriptions[experiment_version].append(line.split("    "))
                    
        print(experiment_version, len(gold_descriptions[experiment_version]))
                
gold_descriptions

task2_gpt4_p1 16
task2_gpt4_p2 16
task2_gpt3.5_p1 6
task2_gpt3.5_p2 12
task2_gpt3.5_p3 9
task2_llama7b_p1 16
task2_llama7b_p2 16
task2_llama7b_p3 16
task2_llama13b_p1 16
task2_llama13b_p2 16
task2_llama13b_p3 16
task2_llama34b_p1 16
task2_llama34b_p2 16
task2_llama34b_p3 16


defaultdict(list,
            {'task2_gpt4_p1': [['dataset16',
               "This dataset provides detailed wind measurements, focusing on wind speed, direction, and variability. Structured in a tabular format with columns: ProgramYear, ReportType, CEID, CEName, TypeOfAgency, TypeOfOrg, ESC, TDARegion, SiteID, SiteName, CountyDistrictCode, CECounty, SiteCounty, RuralOrUrbanCode, ClaimDate, CombinedClaim, BreakfastDays, Breakfast, SecondBreakfast, BreakfastTotal, AMSnackDays, AMSnack, SecondAMSnack, AMSnackTotal, LunchDays, Lunch, SecondLunch, LunchTotal, PMSnackDays, PMSnack, SecondPMSnack, PMSnackTotal, SupperDays, Supper, SecondSupper, SupperTotal. The data types include numerical and date-time formats. The 'Time Stamp' indicates the measurement time, 'Average Wind Speed' and 'Standard Deviation' quantify wind speed and variability, and 'Average Wind Direction' provides directional data. Values include time (in MM/DD/YYYY HH:MM format), wind speed and standard deviation (numerical)

In [8]:
gold_descriptions.keys()

dict_keys(['task2_gpt4_p1', 'task2_gpt4_p2', 'task2_gpt3.5_p1', 'task2_gpt3.5_p2', 'task2_gpt3.5_p3', 'task2_llama7b_p1', 'task2_llama7b_p2', 'task2_llama7b_p3', 'task2_llama13b_p1', 'task2_llama13b_p2', 'task2_llama13b_p3', 'task2_llama34b_p1', 'task2_llama34b_p2', 'task2_llama34b_p3'])

In [9]:
title_descrip_rel = [(qr)+(meta_1st[qr[0].split('/')[0]]['title'], meta_1st[qr[0].split('/')[0]]['notes']) for qr in query_rel_list]

Q = 'Q1'
irrelevant_title_descrip = [i[2:] for i in title_descrip_rel if i[1]==Q and i[2]==0]
print(len(irrelevant_title_descrip))

# relevant_ids = [k for k in dataset_title if dataset_title[k][0]==Q]
# print(relevant_ids)

# gold_descrip_key = "task1_gpt4_p1"



964


In [10]:
relevant_id_title_descrip = [(i[0], dataset_title[i[0]][1], dataset_title[i[0]][2], i[1]) for i in gold_descriptions[experiment_version]]
relevant_id_title_descrip[0]



('dataset14',
 'School Nutrition Programs - Meal Reimbursement Information - Program Year 2022-2023',
 'This dataset contains claims information for paid or approved to pay meal reimbursement for sites participating in school nutrition programs for program year 2022-2023.',
 "The dataset is about school nutrition programs' meal reimbursement information in program year 2022-2023. The dataset has 15 columns, including ProgramYear, ReportType, CEID, CEName, TypeOfAgency, TypeOfOrg, ESC, TDARegion, SiteID, SiteName, CountyDistrictCode, CECounty, SiteCounty, ClaimDate, EnrollmentQty, FreeEligQty, RedcEligQty, PaidEligQty, BreakfastDays, BreakfastTotal, BreakfastADP, BreakfastServedFree, BreakfastServedRedc, BreakfastServedPaid, LunchDays, LunchTotal, LunchADP, LunchServedFree, LunchServedRedc, LunchServedPaid, SnackDays, SnackTotal, SnackADP, SnacksServedFree, SnacksServedRedc, SnacksServedPaid, TotalMeals_Snacks, MilkDays, Milktotal, MilkADP, MilkServedFree, MilkServedReduced, MilkServedP

In [11]:
from rank_bm25 import BM25Okapi

def compute_ndcg(relevantce_true, relevantce_test, k):
    ideal_dcg = np.sum(relevantce_true / np.log2(np.arange(2, k + 2)))
    dcg = np.sum(relevantce_test / np.log2(np.arange(2, k + 2)))
    ndcg = dcg / ideal_dcg
    return ndcg

def downstream_task(documents, query, relevances, ks, debug=False):
    tokenized_corpus = [doc.lower().split() for doc in documents]
    bm25 = BM25Okapi(tokenized_corpus)
    tokenized_query = query.lower().split()
    if debug:
        print(tokenized_query)
    
    scores = bm25.get_scores(tokenized_query)
    sorted_indices = np.argsort(scores)[::-1]
    sorted_rel_true = sorted(relevances, reverse=True)
    sorted_rel_test = np.array(relevances)[sorted_indices].tolist()

    results = {}
    for k in ks:
        y_true = [1 if i!=0 else 0 for i in sorted_rel_true[:k]]
        y_pred = [1 if i!=0 else 0 for i in sorted_rel_test[:k]]
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        ndcg = compute_ndcg(sorted_rel_true[:k], sorted_rel_test[:k], k)
        results[k] = {"precision": precision,
                      "recall": recall,
                      "ndcg": ndcg}
        if debug:
            print("======k@", k)
            print("y_true", y_true)
            print("y_pred", y_pred)
            print(sorted_rel_true[:k])
            print(sorted_rel_test[:k])
            print("ndcg", ndcg)
            print("precision", precision)
            print("recall", recall)
    return results

In [22]:
result_table_generated = []
result_table_original = []
result_table_diff = []
for model_v in model_versions:
    for prompt_v in prompt_versions:
        if model_v=="gpt4" and prompt_v=="p3":
            continue
        experiment_version = "_".join([task, model_v, prompt_v])
        print("="*9, experiment_version, "="*9)
        
        relevant_id_title_descrip = [(i[0], dataset_title[i[0]][1], dataset_title[i[0]][2], i[1]) for i in gold_descriptions[experiment_version]]
        relevant_id_title_descrip = sorted(relevant_id_title_descrip)
        print(len(relevant_id_title_descrip))

        relevant_ids = [i[0] for i in relevant_id_title_descrip]
#         relevant_ids = sorted(relevant_ids)
        print(relevant_ids)
        print(len(relevant_ids))

        import random
        random.seed(199)
        selected_relevant_ids_empty = random.sample(relevant_ids, int(0.25 * len(relevant_ids)))
        print(len(selected_relevant_ids_empty))

        score_irrel = [i[0] for i in irrelevant_title_descrip]
        doc_irrel = [i[1]+i[2] for i in irrelevant_title_descrip]
        ks = [i for i in range(5,25,5)]
#         query = "wind speed Kanasas 2003"
#         query = "national school lunch program how many meal served"
        query = "Schools who have participated in the National School Lunch Program"

        score_rel = [dataset_title[i[0]][-1] for i in relevant_id_title_descrip]
        doc_rel = [i[1]+i[3] for i in relevant_id_title_descrip]
        res_generated = downstream_task(doc_irrel+doc_rel, query, score_irrel+score_rel, ks, debug=True)
        result_table_generated.append({
            "Method":experiment_version,
            "Number of Datasets":len(gold_descriptions[experiment_version]),
            "NDCG@5":res_generated[5]['ndcg'],
            "NDCG@10":res_generated[10]['ndcg'],
            "NDCG@15":res_generated[15]['ndcg'],
            "NDCG@20":res_generated[20]['ndcg'],
            "Precision@5":res_generated[5]['precision'],
            "Precision@10":res_generated[10]['precision'],
            "Precision@15":res_generated[15]['precision'],
            "Precision@20":res_generated[20]['precision'],
            "Recall@5":res_generated[5]['recall'],
            "Recall@10":res_generated[10]['recall'],
            "Recall@15":res_generated[15]['recall'],
            "Recall@20":res_generated[20]['recall'],
        })

        score_rel = [dataset_title[i[0]][-1] for i in relevant_id_title_descrip]
        doc_rel = [i[1]+i[2] for i in relevant_id_title_descrip]
        
        res_original = downstream_task(doc_irrel+doc_rel, query, score_irrel+score_rel, ks, debug=True)
        result_table_original.append({
            "Method":experiment_version,
            "Number of Datasets":len(gold_descriptions[experiment_version]),
            "NDCG@5":res_original[5]['ndcg'],
            "NDCG@10":res_original[10]['ndcg'],
            "NDCG@15":res_original[15]['ndcg'],
            "NDCG@20":res_original[20]['ndcg'],
            "Precision@5":res_original[5]['precision'],
            "Precision@10":res_original[10]['precision'],
            "Precision@15":res_original[15]['precision'],
            "Precision@20":res_original[20]['precision'],
            "Recall@5":res_original[5]['recall'],
            "Recall@10":res_original[10]['recall'],
            "Recall@15":res_original[15]['recall'],
            "Recall@20":res_original[20]['recall'],
        })
        
        result_table_diff.append({
            "Method":experiment_version,
            "Number of Datasets":len(gold_descriptions[experiment_version]),
            "NDCG@5":res_generated[5]['ndcg']-res_original[5]['ndcg'],
            "NDCG@10":res_generated[10]['ndcg']-res_original[10]['ndcg'],
            "NDCG@15":res_generated[15]['ndcg']-res_original[15]['ndcg'],
            "NDCG@20":res_generated[20]['ndcg']-res_original[20]['ndcg'],
            "Precision@5":res_generated[5]['precision']-res_original[5]['precision'],
            "Precision@10":res_generated[10]['precision']-res_original[10]['precision'],
            "Precision@15":res_generated[15]['precision']-res_original[15]['precision'],
            "Precision@20":res_generated[20]['precision']-res_original[20]['precision'],
            "Recall@5":res_generated[5]['recall']-res_original[5]['recall'],
            "Recall@10":res_generated[10]['recall']-res_original[10]['recall'],
            "Recall@15":res_generated[15]['recall']-res_original[15]['recall'],
            "Recall@20":res_generated[20]['recall']-res_original[20]['recall'],
        })
        

========= task2_gpt4_p1 =========
16
['dataset1', 'dataset10', 'dataset11', 'dataset12', 'dataset13', 'dataset14', 'dataset15', 'dataset16', 'dataset2', 'dataset3', 'dataset4', 'dataset5', 'dataset6', 'dataset7', 'dataset8', 'dataset9']
16
4
['schools', 'who', 'have', 'participated', 'in', 'the', 'national', 'school', 'lunch', 'program']
======k@ 5
y_true [1, 1, 1, 1, 1]
y_pred [0, 0, 1, 1, 0]
[1, 1, 1, 1, 1]
[0.0, 0.0, 1.0, 1.0, 0.0]
ndcg 0.3156484524795145
precision 1.0
recall 0.4
======k@ 10
y_true [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y_pred [0, 0, 1, 1, 0, 0, 0, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
ndcg 0.3405196929097572
precision 1.0
recall 0.4
======k@ 15
y_true [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y_pred [0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
ndcg 0.31155194620640786
precision 1.0
recall 0.33333

['schools', 'who', 'have', 'participated', 'in', 'the', 'national', 'school', 'lunch', 'program']
======k@ 5
y_true [1, 1, 1, 1, 1]
y_pred [1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1.0, 1.0, 1.0, 1.0, 1.0]
ndcg 1.0
precision 1.0
recall 1.0
======k@ 10
y_true [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
y_pred [1, 1, 1, 1, 1, 1, 0, 0, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0]
ndcg 0.8475028682434304
precision 1.0
recall 0.7777777777777778
======k@ 15
y_true [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
y_pred [1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]
ndcg 0.9727557479898327
precision 0.7777777777777778
recall 0.7777777777777778
======k@ 20
y_true [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
y_pred [1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0

['schools', 'who', 'have', 'participated', 'in', 'the', 'national', 'school', 'lunch', 'program']
======k@ 5
y_true [1, 1, 1, 1, 1]
y_pred [1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1.0, 1.0, 1.0, 1.0, 1.0]
ndcg 1.0
precision 1.0
recall 1.0
======k@ 10
y_true [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y_pred [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ndcg 0.6489315753318466
precision 1.0
recall 0.5
======k@ 15
y_true [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y_pred [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]
ndcg 0.595434975154242
precision 1.0
recall 0.4666666666666667
======k@ 20
y_true [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
y_pred [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 0.0,

['schools', 'who', 'have', 'participated', 'in', 'the', 'national', 'school', 'lunch', 'program']
======k@ 5
y_true [1, 1, 1, 1, 1]
y_pred [1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1.0, 1.0, 1.0, 1.0, 1.0]
ndcg 1.0
precision 1.0
recall 1.0
======k@ 10
y_true [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y_pred [1, 1, 1, 1, 1, 1, 0, 0, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0]
ndcg 0.793584067764911
precision 1.0
recall 0.7
======k@ 15
y_true [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
y_pred [1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0]
ndcg 0.7475912507452882
precision 1.0
recall 0.6666666666666666
======k@ 20
y_true [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
y_pred [1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [21]:
len(gold_descriptions[experiment_version])

16

In [14]:
columns = list(result_table_original[0].keys())
df_generated = pd.DataFrame(columns=columns)
for record in result_table_generated:
    df_generated = df_generated.append(record, ignore_index=True)

df_original = pd.DataFrame(columns=columns)
for record in result_table_original:
    df_original = df_original.append(record, ignore_index=True)
    
df_diff = pd.DataFrame(columns=columns)
for record in result_table_diff:
    df_diff = df_diff.append(record, ignore_index=True)

In [15]:
print(task)

task2


In [16]:
# df_generated.to_csv(task+"_ranking_generated_description.csv", index=False)
df_generated

,Method,Number of Datasets,NDCG@5,NDCG@10,NDCG@15,NDCG@20,Precision@5,Precision@10,Precision@15,Precision@20,Recall@5,Recall@10,Recall@15,Recall@20
0,task2_gpt4_p1,16,0.315648,0.340520,0.311552,0.299069,1.0,1.0,1.000000,1.000000,0.4,0.400000,0.333333,0.312500
1,task2_gpt4_p2,16,0.315648,0.340520,0.311552,0.299069,1.0,1.0,1.000000,1.000000,0.4,0.400000,0.333333,0.312500
2,task2_gpt3.5_p1,6,0.868795,0.775148,0.775148,0.775148,1.0,1.0,1.000000,1.000000,0.8,0.666667,0.666667,0.666667
3,task2_gpt3.5_p2,12,1.000000,0.727330,0.700471,0.700471,1.0,1.0,0.857143,0.857143,1.0,0.600000,0.500000,0.500000
4,task2_gpt3.5_p3,9,1.000000,0.693022,0.753184,0.808516,1.0,1.0,0.833333,0.714286,1.0,0.555556,0.555556,0.555556
5,task2_llama7b_p1,16,1.000000,1.000000,0.817825,0.785057,1.0,1.0,1.000000,1.000000,1.0,1.000000,0.733333,0.687500
6,task2_llama7b_p2,16,1.000000,0.870125,0.719308,0.845669,1.0,1.0,1.000000,0.769231,1.0,0.800000,0.600000,0.625000
7,task2_llama7b_p3,16,1.000000,0.870125,0.764740,0.771385,1.0,1.0,1.000000,0.909091,1.0,0.800000,0.666667,0.625000
8,task2_llama13b_p1,16,1.000000,0.648932,0.639104,0.652050,1.0,1.0,1.000000,0.888889,1.0,0.500000,0.533333,0.500000
9,task2_llama13b_p2,16,1.000000,0.648932,0.595435,0.611645,1.0,1.0,1.000000,1.000000,1.0,0.500000,0.466667,0.500000


In [17]:
# df_original.to_csv(task+"_ranking_original_description.csv", index=False)
df_original

,Method,Number of Datasets,NDCG@5,NDCG@10,NDCG@15,NDCG@20,Precision@5,Precision@10,Precision@15,Precision@20,Recall@5,Recall@10,Recall@15,Recall@20
0,task2_gpt4_p1,16,1.0,0.793584,0.747591,0.835533,1.0,1.000000,1.000000,0.846154,1.0,0.700000,0.666667,0.687500
1,task2_gpt4_p2,16,1.0,0.793584,0.747591,0.835533,1.0,1.000000,1.000000,0.846154,1.0,0.700000,0.666667,0.687500
2,task2_gpt3.5_p1,6,1.0,0.987671,0.987671,0.987671,1.0,0.833333,0.833333,0.833333,1.0,0.833333,0.833333,0.833333
3,task2_gpt3.5_p2,12,1.0,0.793584,0.862903,0.909992,1.0,1.000000,0.800000,0.727273,1.0,0.700000,0.666667,0.666667
4,task2_gpt3.5_p3,9,1.0,0.847503,0.972756,0.972756,1.0,1.000000,0.777778,0.777778,1.0,0.777778,0.777778,0.777778
5,task2_llama7b_p1,16,1.0,0.793584,0.747591,0.835533,1.0,1.000000,1.000000,0.846154,1.0,0.700000,0.666667,0.687500
6,task2_llama7b_p2,16,1.0,0.793584,0.747591,0.835533,1.0,1.000000,1.000000,0.846154,1.0,0.700000,0.666667,0.687500
7,task2_llama7b_p3,16,1.0,0.793584,0.747591,0.835533,1.0,1.000000,1.000000,0.846154,1.0,0.700000,0.666667,0.687500
8,task2_llama13b_p1,16,1.0,0.793584,0.747591,0.835533,1.0,1.000000,1.000000,0.846154,1.0,0.700000,0.666667,0.687500
9,task2_llama13b_p2,16,1.0,0.793584,0.747591,0.835533,1.0,1.000000,1.000000,0.846154,1.0,0.700000,0.666667,0.687500


In [18]:
# df_diff.to_csv(task+"_ranking_diff_positive-means-improve-with-description.csv", index=False)
df_diff

,Method,Number of Datasets,NDCG@5,NDCG@10,NDCG@15,NDCG@20,Precision@5,Precision@10,Precision@15,Precision@20,Recall@5,Recall@10,Recall@15,Recall@20
0,task2_gpt4_p1,16,-0.684352,-0.453064,-0.436039,-0.536464,0.0,0.000000,0.000000,0.153846,-0.6,-0.300000,-0.333333,-0.375000
1,task2_gpt4_p2,16,-0.684352,-0.453064,-0.436039,-0.536464,0.0,0.000000,0.000000,0.153846,-0.6,-0.300000,-0.333333,-0.375000
2,task2_gpt3.5_p1,6,-0.131205,-0.212523,-0.212523,-0.212523,0.0,0.166667,0.166667,0.166667,-0.2,-0.166667,-0.166667,-0.166667
3,task2_gpt3.5_p2,12,0.000000,-0.066254,-0.162432,-0.209522,0.0,0.000000,0.057143,0.129870,0.0,-0.100000,-0.166667,-0.166667
4,task2_gpt3.5_p3,9,0.000000,-0.154481,-0.219572,-0.164240,0.0,0.000000,0.055556,-0.063492,0.0,-0.222222,-0.222222,-0.222222
5,task2_llama7b_p1,16,0.000000,0.206416,0.070234,-0.050476,0.0,0.000000,0.000000,0.153846,0.0,0.300000,0.066667,0.000000
6,task2_llama7b_p2,16,0.000000,0.076541,-0.028283,0.010136,0.0,0.000000,0.000000,-0.076923,0.0,0.100000,-0.066667,-0.062500
7,task2_llama7b_p3,16,0.000000,0.076541,0.017149,-0.064148,0.0,0.000000,0.000000,0.062937,0.0,0.100000,0.000000,-0.062500
8,task2_llama13b_p1,16,0.000000,-0.144652,-0.108487,-0.183483,0.0,0.000000,0.000000,0.042735,0.0,-0.200000,-0.133333,-0.187500
9,task2_llama13b_p2,16,0.000000,-0.144652,-0.152156,-0.223888,0.0,0.000000,0.000000,0.153846,0.0,-0.200000,-0.200000,-0.187500


In [19]:
a = [1,1,1]
b = [1,1,0]
precision_score(a,b)

1.0